In [2]:
import pandas as pd
import numpy as np
import re
import ast
import os
from scipy.stats import binom
from sentence_transformers import SentenceTransformer, util
from collections import defaultdict

embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

In [7]:
# Implement Wilson Score Interval to correct ranking
def wilson_score(rating, num_reviews, confidence=1.9):
    """Computes Wilson Score for a course based on its rating and number of reviews."""
    if num_reviews == 0:
        return 0  # If no reviews, lowest possible score

    p_hat = rating / 5  # Convert rating scale to [0,1]
    n = num_reviews

    denominator = 1 + (confidence ** 2 / n)
    center_adj = p_hat + (confidence ** 2 / (2 * n))
    margin = confidence * ((p_hat * (1 - p_hat) / n) + (confidence ** 2 / (4 * n ** 2))) ** 0.5

    wilson_lower_bound = (center_adj - margin) / denominator
    return wilson_lower_bound


In [5]:
# Set your data directory
data_dir = "./udemy_data"

# Dictionary to collect all DataFrames per job title
job_title_dfs = defaultdict(list)

# Loop through all files
for filename in os.listdir(data_dir):
    if filename.endswith(".csv"):
        # Extract job title from file name
        job_title = filename.split("_")[0]  # or use a better parser if needed

        # Read CSV
        file_path = os.path.join(data_dir, filename)
        try:
            df = pd.read_csv(file_path)
            if df.empty:
                print(f"Skipping empty file: {filename}")
                continue
        except pd.errors.EmptyDataError:
            print(f"Skipping invalid or empty file: {filename}")
            continue

        # Append to the job_title's list
        job_title_dfs[job_title].append(df)

# Combine and save for each job title
output_dir = "./udemy_df"
os.makedirs(output_dir, exist_ok=True)

for job_title, dfs in job_title_dfs.items():
    combined_df = pd.concat(dfs, ignore_index=True)
    output_path = os.path.join(output_dir, f"{job_title}.csv")
    combined_df.to_csv(output_path, index=False)



Skipping invalid or empty file: DevOps Engineer_unix_linux.csv
Skipping invalid or empty file: Java Developer_unix.csv
Skipping invalid or empty file: Project Manager_linux operating system.csv
Skipping invalid or empty file: Project Manager_machine learning.csv
Skipping invalid or empty file: Software Development Engineer in Test (SDET)_db2.csv
Skipping invalid or empty file: Software Development Engineer in Test (SDET)_windows server.csv
Skipping invalid or empty file: Software Engineer_ms sql server.csv
Skipping invalid or empty file: Web Developer_api's.csv
Skipping invalid or empty file: Web Developer_restful api's.csv


In [9]:
# Coursera Data Cleaning and None Values Handling

data_dir = "./coursera_df"

# Loop through all files
for filename in os.listdir(data_dir):
    if filename.endswith(".csv"):
        
        job_title = filename.split(".")[0]

        df_coursera = pd.read_csv(f"{data_dir}/{filename}")

        def convert_to_hours(duration):
            duration = duration.lower().strip()  # Normalize text
            
            # Match different patterns
            match = re.search(r'(\d+)\s*-\s*(\d+)\s*(weeks|months|years|meses|hours)', duration)
            single_match = re.search(r'(\d+)\s*(weeks|months|years|meses|hours)', duration)
            # Handle ranges
            if match:
                num1, num2, unit = int(match.group(1)), int(match.group(2)), match.group(3)
            elif single_match:
                num1, num2, unit = int(single_match.group(1)), int(single_match.group(1)), single_match.group(2)
            else:
                return None
            # Convert to days
            if "week" in unit:
                hours = 10 * num1
            elif "month" in unit or "mes" in unit:
                hours = 10 * num1 * 4
            elif "year" in unit:
                hours = 10 * num1 * 52
            elif "hour" in unit:
                hours = num1
            return hours

        # Function to safely convert string lists to actual Python lists
        def str_to_list(value):
            if isinstance(value, str):
                return ast.literal_eval(value)  # Convert string to list
            else:
                return []  # Handle NaN or other types by returning an empty list

        df_coursera["course_skill"] = df_coursera["course_skill"].apply(str_to_list)

        # Change num_review to int type
        df_coursera["num_review"] = df_coursera["num_review"].fillna(0).astype(int)

        # Map difficulty to numeric
        difficulty_mapping = {
            'Beginner': 1,
            'Intermediate': 2,
            'Mixed': 1.5,
            'Advanced': 3,
            'All Levels': 1.5
        }
        df_coursera['difficulty'] = df_coursera['difficulty'].fillna("Mixed")
        df_coursera['difficulty_numeric'] = df_coursera['difficulty'].fillna("Mixed").map(difficulty_mapping)

        course_type_str = ["Course", "Professional Certificate"]
        mask = ~df_coursera['course_type'].isin(course_type_str)
        # Change incorrect course_type to Course
        df_coursera.loc[mask, 'course_type'] = "Course"
        df_coursera.loc[df_coursera['course_type'] == "Professional Certificate", 'course_type'] = "Certificate"
        df_coursera.loc[df_coursera['course_type'] == "Certificate", 'difficulty_numeric'] = 3.0


        # Convert duration string into min max int in days
        df_coursera['duration'] = df_coursera['duration'].fillna(df_coursera['duration'].mode()[0])
        df_coursera['duration'] = df_coursera['duration'].apply(convert_to_hours)

        df_coursera["price"] = df_coursera["duration"] * 59/30/5

        # Fill None course skill with skill
        df_coursera["description"] = df_coursera["course_skill"].fillna(df_coursera["skill"])

        # Fill None rating with average rating
        df_coursera["rating"] = df_coursera["rating"].fillna(df_coursera["rating"].mean())
        df_coursera["wilson_score"] = df_coursera.apply(lambda row: wilson_score(row["rating"], row["num_review"]), axis=1)

        df_coursera["description"] = df_coursera["description"].apply(lambda x: "list " + str(x).replace("[", "").replace("]", "").replace("'", "").strip())

        df_coursera = df_coursera.drop(columns = ["course_skill", "difficulty"])
        # Output folder for saving HTML files
        data_folder = "coursera_cleaned_df"
        os.makedirs(data_folder, exist_ok=True)  # Ensure folder exists

        df_coursera.to_csv(f"{data_folder}/{filename}", index=False)


In [11]:
# Udemy Data Cleaning
# Coursera Data Cleaning and None Values Handling

data_dir = "./udemy_df"

# Loop through all files
for filename in os.listdir(data_dir):
    if filename.endswith(".csv"):
        
        job_title = filename.split(".")[0]

        df_udemy = pd.read_csv(f"{data_dir}/{filename}")


        # Fill None
        df_udemy["rating"] = df_udemy["rating"].fillna(df_udemy["rating"].mean())
        df_udemy["num_review"] = df_udemy["num_review"].fillna(0)
        df_udemy["difficulty"] = df_udemy["difficulty"].fillna("All Levels")
        df_udemy["num_lecture"] = df_udemy["num_lecture"].fillna(1)
        df_udemy["duration"] = df_udemy["duration"].fillna(df_udemy["duration"].mode()[0])
        df_udemy["price"] = df_udemy["price"].fillna(df_udemy["price"].mode()[0])

        # Data type
        df_udemy["description"] = df_udemy["course_description"].fillna("")
        df_udemy["num_review"] = df_udemy["num_review"].astype(int)
        df_udemy['difficulty_numeric'] = df_udemy['difficulty'].map(difficulty_mapping)
        conversion_factors = {
            'lectures': 1,
            'questions': 0.1,
            'Expert': 10
        }
        conversion_factors = {
            'lectures': 1,
            'questions': 0.1,
            'Expert': 10
        }

        # Extract the numeric part and unit from `num_lecture`
        df_udemy[['num_value', 'unit']] = df_udemy['num_lecture'].str.extract(r'(\d+)\s*([a-zA-Z]+)')
        # Convert `num_value` to numeric, replacing errors with NaN
        df_udemy['num_value'] = pd.to_numeric(df_udemy['num_value'], errors='coerce')
        # Fill missing values in 'num_value' with 10 for Expert
        df_udemy['num_value'] = df_udemy['num_value'].fillna(10)
        # Apply conversion factors safely, ensuring no NaN values
        df_udemy['num_lecture'] = df_udemy.apply(
            lambda row: int(round(row['num_value'] * conversion_factors.get(row['unit'], 1))) if pd.notna(row['num_value']) else 0,
            axis=1
        )

        def is_certificate(title):
            return "Certificate" if isinstance(title, str) and "certif" in title.lower().strip() else "Course"

        df_udemy["course_type"] = df_udemy["title"].apply(is_certificate)
        df_udemy.loc[df_udemy["course_type"] == "Course", "course_type"] = df_udemy.loc[df_udemy["course_type"] == "Course", "course_description"].apply(is_certificate)
        df_udemy.loc[df_udemy["course_type"] == "Certificate", "difficulty_numeric"] = 3.0

        # Extract the numeric value before "total hours"
        df_udemy['duration'] = df_udemy['duration'].str.extract(r'([\d\.]+)')  # Extracts the numeric part
        # Convert to float
        df_udemy['duration'] = pd.to_numeric(df_udemy['duration'], errors='coerce')

        # Ensure 'price' is a string type before extraction
        df_udemy['price'] = df_udemy['price'].astype(str)
        # Extract numeric values from 'price' (handling cases with currency symbols like "US124.99")
        df_udemy['price'] = df_udemy['price'].str.extract(r'([\d]+\.?\d*)')  # Matches numbers with decimals
        # Convert to float
        df_udemy['price'] = pd.to_numeric(df_udemy['price'], errors='coerce')
        df_udemy['price'] = df_udemy['price'].fillna(df_udemy['price'].mean())
        df_udemy["wilson_score"] = df_udemy.apply(lambda row: wilson_score(row["rating"], row["num_review"]), axis=1)

        df_udemy = df_udemy.drop(columns = ['num_value', 'unit', "num_lecture", "course_description", "difficulty"])
        # Output folder for saving HTML files
        data_folder = "udemy_cleaned_df"
        os.makedirs(data_folder, exist_ok=True)  # Ensure folder exists
        df_udemy.to_csv(f"{data_folder}/{filename}", index=False)



In [ ]:
# Textbook Data Cleaning
df_textbook = pd.read_csv("./unprocessed-df/df_textbook.csv").iloc[:, 1:]
# Fill None
df_textbook = df_textbook.dropna(subset=['title'])
df_textbook.loc[:, "author"] = df_textbook["author"].fillna("Unknown")
df_textbook.loc[:, "publisher"] = df_textbook["publisher"].fillna("Unknown")
df_textbook.loc[:, "isbn"] = df_textbook["isbn"].fillna("Unknown")
df_textbook.loc[:, "rating"] = df_textbook["rating"].fillna(df_textbook["rating"].min())
df_textbook.loc[:, "language"] = df_textbook["language"].fillna("Unknown")

# Clean price, num_page, and num_rating, rankings
df_textbook.loc[:, "rankings"] = df_textbook["rankings"].apply(str_to_list)

# Convert 'price' to float
df_textbook['price'] = df_textbook['price'].fillna("$0")
df_textbook['price'] = df_textbook['price'].apply(lambda x: x.split()[0].replace("$", "").replace(",","")).astype(float)

# Calculate mean and standard deviation of existing prices
mean_price = df_textbook['price'].mean()
std_price = df_textbook['price'].std()
# Reduce the variation by scaling down the standard deviation
scaled_std = std_price * 0.3  # Reduce variation (adjust factor as needed)
# Generate different normally distributed values for each missing price
nan_indices = df_textbook['price'].isna()
random_prices = np.random.normal(loc=mean_price, scale=scaled_std, size=nan_indices.sum())
# Ensure non-negative values and round to 2 decimal places
random_prices = np.abs(random_prices)  # Make sure values are non-negative
random_prices = np.round(random_prices, 2)  # Round to 2 decimal places
# Assign generated prices to NaN values in the DataFrame
df_textbook.loc[nan_indices, 'price'] = random_prices

# Extract numeric values from 'num_page'
df_textbook['num_page'] = df_textbook['num_page'].str.extract(r'(\d+)')
# Convert to float first to handle NaN values
df_textbook['num_page'] = pd.to_numeric(df_textbook['num_page'], errors='coerce')
# Fill NaN values with the mean number of pages, then convert to int
df_textbook['num_page'] = df_textbook['num_page'].fillna(df_textbook['num_page'].mean()).astype(int)

def extract_number(r):
    if isinstance(r, str):  # Ensure it's a string
        num_part = r.split()[0]  # Take the first part of the string
        num_part = num_part.replace(",", "")  # Remove commas
        if num_part.isdigit():  # Check if it's now a valid number
            return int(num_part)
    return 0  # Default to 0 for non-numeric values
# Apply function to the column
df_textbook["num_rating"] = df_textbook["num_rating"].apply(extract_number)

df_textbook.to_csv("cleaned-df/df_textbook.csv", index=False)

df_textbook.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5060 entries, 0 to 5165
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   skill       5060 non-null   object 
 1   link        2392 non-null   object 
 2   image_link  5060 non-null   object 
 3   author      5060 non-null   object 
 4   title       5060 non-null   object 
 5   price       5060 non-null   float64
 6   publisher   5060 non-null   object 
 7   language    5060 non-null   object 
 8   num_page    5060 non-null   int32  
 9   isbn        5060 non-null   object 
 10  rankings    5060 non-null   object 
 11  rating      5060 non-null   float64
 12  num_rating  5060 non-null   int64  
dtypes: float64(2), int32(1), int64(1), object(9)
memory usage: 533.7+ KB


In [22]:
# Youtube Data Cleaning
df_youtube = pd.read_csv("./unprocessed-df/df_youtube.csv").iloc[:, 1:]

# Fill None
df_youtube.loc[:, "thumbnail_link"] = df_youtube["thumbnail_link"].fillna("Unknown")
df_youtube.loc[:, "num_views"] = df_youtube["num_views"].fillna("Unknown")
df_youtube.loc[:, "snippet"] = df_youtube["snippet"].fillna("Unknown")
df_youtube.loc[:, "chapter"] = df_youtube["chapter"].fillna(1)
df_youtube.loc[:, 'series'] = df_youtube['series'].apply(str_to_list)
df_youtube.loc[:, 'series'] = df_youtube['series'].apply(lambda x: x if isinstance(x, list) else [])
df_youtube["chapter"] = df_youtube["chapter"].astype(int)

# Clean age, duration, num_views

# Define a function to extract and convert views
def convert_views(value):
    import re
    value = str(value).strip()  # Ensure it's a string and remove extra spaces

    # Check if the value ends with "views"
    if not value.endswith("views"):
        return 0  # If not a valid view count, return 0
    # Extract the numeric part and the suffix (K, M, etc.)
    match = re.match(r'([\d\.]+)([KkMm]?) views', value)
    if match:
        num, suffix = match.groups()
        num = float(num)  # Convert to float for decimal values
        # Convert K (thousands) and M (millions) to proper numbers
        if suffix == 'K' or suffix == 'k':
            return int(num * 1_000)
        elif suffix == 'M' or suffix == 'm':
            return int(num * 1_000_000)
        else:
            return int(num)  # If no suffix, return as integer
    else:
        return 0  # Default case
# Apply function to the column in df_youtube
df_youtube['num_views'] = df_youtube['num_views'].apply(convert_views)

import pandas as pd
import re

# Define function to convert time since posted into years
def convert_to_years(value):
    value = str(value).strip()  # Ensure it's a string and remove extra spaces
    # Check if the value ends with "ago" (valid time indicator)
    if not value.endswith("ago"):
        return None  # If not a valid date format, return None
    # Match the number and time unit
    match = re.match(r'(\d+)\s*(year|minute|hour|day|month)s?\s+ago', value)
    if match:
        num, unit = int(match.group(1)), match.group(2)
        # Convert different time units into years
        if unit == 'year':
            return num
        elif unit == 'month':
            return round(num / 12)  # Approximate months to years
        elif unit == 'day':
            return round(num / 365)  # Approximate days to years
        elif unit == 'hour':
            return 0  # Hours are within the same year
        elif unit == 'minute':
            return 0  # Minutes are within the same year
    return 0  # Default to 0 years for unhandled cases
# Apply function to the column in df_youtube
df_youtube['age'] = df_youtube['age'].apply(convert_to_years)
df_youtube['age'] = df_youtube['age'].fillna(0)

# Define function to convert duration into seconds
def convert_to_seconds(value):
    value = str(value).strip()  # Ensure it's a string and remove extra spaces
    # Handle special cases (SHORTS, UPCOMING, and irregular formats)
    if value in ["SHORTS", "UPCOMING"]:
        return 3 * 60  # Assume 3 minutes (180 seconds) for Shorts and Upcoming videos
    # Match durations in HH:MM:SS, MM:SS, or SS formats
    match = re.match(r'(?:(\d+):)?(\d+):(\d+)$', value)  # Matches HH:MM:SS or MM:SS
    if match:
        hours = int(match.group(1)) if match.group(1) else 0
        minutes = int(match.group(2))
        seconds = int(match.group(3))
        return hours * 3600 + minutes * 60 + seconds  # Convert to total seconds
    return 3 * 60  # Default to 3 minutes if format is unrecognized

# Apply function to the duration column in df_youtube
df_youtube['duration_seconds'] = df_youtube['duration'].apply(convert_to_seconds)
df_youtube = df_youtube.drop(columns=["duration"])

df_youtube.to_csv("cleaned-df/df_youtube.csv", index=False)

df_youtube.info()

FileNotFoundError: [Errno 2] No such file or directory: './unprocessed-df/df_youtube.csv'